In [ ]:
import os
import sys
import json

# Config
os.environ["STEAM__API_KEY"] = "id"
os.environ["FABRIC__WORKSPACE_ID"] = "id"
os.environ["FABRIC__BRONZE_LAKEHOUSE_ID"] = "id"
os.environ["FABRIC__SILVER_LAKEHOUSE_ID"] = "id"
os.environ["FABRIC__GOLD_LAKEHOUSE_ID"] = "id"

In [ ]:
sys.path.insert(0, "/lakehouse/default/Files")

In [ ]:
from src.steam_analytics.config import Settings

settings = Settings()
BRONZE_PATH = settings.fabric.bronze_abfss_path
SILVER_PATH = settings.fabric.silver_abfss_path

In [ ]:
from src.steam_analytics.transformation.silver import run_silver_transform

results = run_silver_transform(spark, BRONZE_PATH, SILVER_PATH)

In [ ]:
from notebookutils import mssparkutils
import json

mssparkutils.notebook.exit(json.dumps({
    "status": "success" if results.all_successful else "partial",
    "total_processed": results.total_processed,
    "total_inserted": results.total_inserted,
    "total_updated": results.total_updated,
}))

## Verify Results

In [ ]:
# Check dim_games
print("dim_games (Current Records):")
df_games = spark.read.format("delta").load(f"{SILVER_PATH}/Tables/dim_games")
df_games.filter("is_current = true").select(
    "app_id", "name", "price_final_cents", "valid_from", "is_current"
).show(10, truncate=False)

print(f"\nTotal records: {df_games.count()}")
print(f"Current records: {df_games.filter('is_current = true').count()}")
print(f"Historical records: {df_games.filter('is_current = false').count()}")

In [ ]:
# Check dim_game_reviews
print("dim_game_reviews (Current Records):")
df_reviews = spark.read.format("delta").load(f"{SILVER_PATH}/Tables/dim_game_reviews")
df_reviews.filter("is_current = true").select(
    "app_id", "total_reviews", "positive_ratio", "review_score_desc", "valid_from"
).show(10, truncate=False)

In [ ]:
# Check fact_player_counts
print("fact_player_counts:")
df_players = spark.read.format("delta").load(f"{SILVER_PATH}/Tables/fact_player_counts")
df_players.select(
    "app_id", "player_count", "snapshot_timestamp", "snapshot_date"
).orderBy("snapshot_timestamp", ascending=False).show(10, truncate=False)